# Action Rules Example

### Import Module

In [1]:
from action_rules import ActionRules

### Import Data

In [2]:
import pandas as pd
# Data
transactions = {'Sex': ['M', 'F', 'M', 'M', 'F', 'M', 'F'], 
                'Age': ['Y', 'Y', 'O', 'Y', 'Y', 'O', 'Y'],
                'Class': ['1', '1', '2', '2', '1', '1', '2'],
                'Embarked': ['S', 'C', 'S', 'C', 'S', 'C', 'C'],
                'Survived': ['1', '1', '0', '0', '1', '1', '0'],
               }
data = pd.DataFrame.from_dict(transactions)

### Initialize ActionRules Miner with Parameters

In [3]:
# Parameters
stable_attributes = ['Age', 'Sex']
flexible_attributes = ['Embarked', 'Class']
target = 'Survived'
min_stable_attributes = 2
min_flexible_attributes = 1 #min 1
min_undesired_support = 1
min_undesired_confidence = 0.5 #min 0.5
min_desired_support = 1
min_desired_confidence = 0.5 #min 0.5
undesired_state = '0'
desired_state = '1'
# Action Rules Mining
action_rules = ActionRules(
    min_stable_attributes=min_stable_attributes,
    min_flexible_attributes=min_flexible_attributes,
    min_undesired_support=min_undesired_support,
    min_undesired_confidence=min_undesired_confidence,
    min_desired_support=min_desired_support,
    min_desired_confidence=min_desired_confidence,
    verbose=True
)

### Train the Model

Use GPU acceleration

In [4]:
! nvidia-smi

Fri Jul 19 11:43:49 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L40S                    Off | 00000000:03:00.0 Off |                    0 |
| N/A   48C    P0              82W / 350W |    454MiB / 46068MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
action_rules.fit(
    data=data,
    stable_attributes=stable_attributes,
    flexible_attributes=flexible_attributes,
    target=target,
    target_undesired_state=undesired_state,
    target_desired_state=desired_state,
    use_sparse_matrix=True,
    use_gpu=False,
)

Maximum number of nodes to check for support:
_____________________________________________
80

SUPPORT for: (0,)
_________________________________________________
- extended by stable attribute
undesired state support: 1
desired state support: 1

SUPPORT for: (1,)
_________________________________________________
- extended by stable attribute
undesired state support: 2
desired state support: 3

SUPPORT for: (0, 2)
_________________________________________________
- extended by stable attribute
undesired state support: 0
desired state support: 0

SUPPORT for: (0, 3)
_________________________________________________
- extended by stable attribute
undesired state support: 1
desired state support: 1

SUPPORT for: (1, 2)
_________________________________________________
- extended by stable attribute
undesired state support: 1
desired state support: 2

SUPPORT for: (1, 3)
_________________________________________________
- extended by stable attribute
undesired state support: 1
desired st

### Print Action Rules

In [6]:
len(action_rules.get_rules().get_ar_notation())

11

In [7]:
# Print rules
for action_rule in action_rules.get_rules().get_ar_notation():
    print(action_rule)

[(Age: O) ∧ (Sex: M) ∧ (Embarked: S → C)] ⇒ [Survived: 0 → 1], support of undesired part: 1, confidence of undesired part: 1.0, support of desired part: 1, confidence of desired part: 1.0, uplift: 1.0
[(Age: O) ∧ (Sex: M) ∧ (Class: 2 → 1)] ⇒ [Survived: 0 → 1], support of undesired part: 1, confidence of undesired part: 1.0, support of desired part: 1, confidence of desired part: 1.0, uplift: 1.0
[(Age: Y) ∧ (Sex: F) ∧ (Embarked*: C)] ⇒ [Survived: 0 → 1], support of undesired part: 1, confidence of undesired part: 0.5, support of desired part: 1, confidence of desired part: 0.5, uplift: 0.0
[(Age: Y) ∧ (Sex: F) ∧ (Embarked: C → S)] ⇒ [Survived: 0 → 1], support of undesired part: 1, confidence of undesired part: 0.5, support of desired part: 1, confidence of desired part: 1.0, uplift: 1.0
[(Age: Y) ∧ (Sex: F) ∧ (Class: 2 → 1)] ⇒ [Survived: 0 → 1], support of undesired part: 1, confidence of undesired part: 1.0, support of desired part: 2, confidence of desired part: 1.0, uplift: 1.0
[(Ag

In [8]:
for action_rule in action_rules.get_rules().get_pretty_ar_notation():
    print(action_rule)

If attribute 'Age' is 'O', attribute 'Sex' is 'M', attribute 'Embarked' value 'S' is changed to 'C', then 'Survived' value '0' is changed to '1 with uplift: 1.0.
If attribute 'Age' is 'O', attribute 'Sex' is 'M', attribute 'Class' value '2' is changed to '1', then 'Survived' value '0' is changed to '1 with uplift: 1.0.
If attribute 'Age' is 'Y', attribute 'Sex' is 'F', attribute (flexible is used as stable) 'Embarked' is 'C', then 'Survived' value '0' is changed to '1 with uplift: 0.0.
If attribute 'Age' is 'Y', attribute 'Sex' is 'F', attribute 'Embarked' value 'C' is changed to 'S', then 'Survived' value '0' is changed to '1 with uplift: 1.0.
If attribute 'Age' is 'Y', attribute 'Sex' is 'F', attribute 'Class' value '2' is changed to '1', then 'Survived' value '0' is changed to '1 with uplift: 1.0.
If attribute 'Age' is 'Y', attribute 'Sex' is 'M', attribute 'Embarked' value 'C' is changed to 'S', then 'Survived' value '0' is changed to '1 with uplift: 1.0.
If attribute 'Age' is 'Y',

In [9]:
export = action_rules.get_rules().get_export_notation()
print(export)

[{"stable": [{"attribute": "Age", "value": "O"}, {"attribute": "Sex", "value": "M"}], "flexible": [{"attribute": "Embarked", "undesired": "S", "desired": "C"}], "target": {"attribute": "Survived", "undesired": "0", "desired": "1"}, "support of undesired part": 1, "confidence of undesired part": 1.0, "support of desired part": 1, "confidence of desired part": 1.0, "uplift": 1.0}, {"stable": [{"attribute": "Age", "value": "O"}, {"attribute": "Sex", "value": "M"}], "flexible": [{"attribute": "Class", "undesired": "2", "desired": "1"}], "target": {"attribute": "Survived", "undesired": "0", "desired": "1"}, "support of undesired part": 1, "confidence of undesired part": 1.0, "support of desired part": 1, "confidence of desired part": 1.0, "uplift": 1.0}, {"stable": [{"attribute": "Age", "value": "Y"}, {"attribute": "Sex", "value": "F"}, {"attribute": "Embarked", "value": "C", "flexible_as_stable": true}], "flexible": [], "target": {"attribute": "Survived", "undesired": "0", "desired": "1"},

## CLI

In [10]:
data.to_csv('data.csv')

### Help

In [11]:
! action-rules --help

/bin/bash: line 1: action-rules: command not found


### Command

In [12]:
! action-rules \
--min_stable_attributes 2 \
--min_flexible_attributes 1 \
--min_undesired_support 1 \
--min_undesired_confidence 0.5 \
--min_desired_support 1 \
--min_desired_confidence 0.5 \
--csv_path "data.csv" \
--stable_attributes "Sex, Age" \
--flexible_attributes "Class, Embarked" \
--target "Survived" \
--undesired_state "0" \
--desired_state "1" \
--output_json_path "output.json" \
--use_gpu 0

/bin/bash: line 1: action-rules: command not found


## Import Rules

In [13]:
from action_rules.input import Input

input_rules = Input()
output = input_rules.import_action_rules(export)
for action_rule in output.get_ar_notation():
    print(action_rule)

[(Age: O) ∧ (Sex: M) ∧ (Embarked: S → C)] ⇒ [Survived: 0 → 1], support of undesired part: 1, confidence of undesired part: 1.0, support of desired part: 1, confidence of desired part: 1.0, uplift: 1.0
[(Age: O) ∧ (Sex: M) ∧ (Class: 2 → 1)] ⇒ [Survived: 0 → 1], support of undesired part: 1, confidence of undesired part: 1.0, support of desired part: 1, confidence of desired part: 1.0, uplift: 1.0
[(Age: Y) ∧ (Sex: F) ∧ (Embarked*: C)] ⇒ [Survived: 0 → 1], support of undesired part: 1, confidence of undesired part: 0.5, support of desired part: 1, confidence of desired part: 0.5, uplift: 0.0
[(Age: Y) ∧ (Sex: F) ∧ (Embarked: C → S)] ⇒ [Survived: 0 → 1], support of undesired part: 1, confidence of undesired part: 0.5, support of desired part: 1, confidence of desired part: 1.0, uplift: 1.0
[(Age: Y) ∧ (Sex: F) ∧ (Class: 2 → 1)] ⇒ [Survived: 0 → 1], support of undesired part: 1, confidence of undesired part: 1.0, support of desired part: 2, confidence of desired part: 1.0, uplift: 1.0
[(Ag